In [1]:
!pip3 install git+https://github.com/tomquirk/linkedin-api.git
!pip install requests
!pip install pandas
!pip install geopandas

  Cloning https://github.com/tomquirk/linkedin-api.git to /tmp/pip-req-build-0k2j6edr
  Running command git clone -q https://github.com/tomquirk/linkedin-api.git /tmp/pip-req-build-0k2j6edr
  Created wheel for linkedin-api: filename=linkedin_api-2.0.0a5-py3-none-any.whl size=22146 sha256=c6f8b98383216a493609b44a39ce39c7f483038a7ebe96884672f9c8cf566b24
  Stored in directory: /tmp/pip-ephem-wheel-cache-otq_iqiu/wheels/31/8e/e2/2b5c53758680b6328eeb5dee5a1ebeed25dfacfbdb19847e15
Successfully built linkedin-api
     |████████████████████████████████| 1.0 MB 5.9 MB/s 
     |████████████████████████████████| 16.7 MB 174 kB/s 
     |████████████████████████████████| 6.3 MB 21.8 MB/s 


In [2]:
from linkedin_api import Linkedin
import linkedin_api
import pandas as pd
import json
import time
import numpy as np
import geopandas as gpd

# linkedin

Set up LinkedIn Account which is especially for this project. To successfully set this up in colab, need to run this in local environment first.

In [105]:
# Authenticate using any Linkedin account credentials
# api = Linkedin('lanxiao@design.upenn.edu', 'Buzhongyao123!')
api = Linkedin('lanxiao1120@yeah.net', 'linkedin611')

Function to get job post.

In [4]:
def get_jobPost_by_SatePosition(position, state):
  response = api.search_jobs(keywords=position,
                            #listed_at=604800,
                            experience=["2"],
                            job_type=["F"],
                            location_name=state + ", United States")

  df = pd.json_normalize(response)
  return(df)


Function to query company names with urn.

In [95]:
def get_company_info(row):
  # subtract company uniqueid
  id = row['companyDetails.company']
  if pd.isna(id) == False:
    id = int(id[29:])
    # inquery company info with id using api
    company = api.get_company(id)
    # maitain name and location info and transform to df
    companyInfo = {}
    companyInfo['name'] = company['name']
    companyInfo['entityUrn'] = company['entityUrn']
    if bool(company['confirmedLocations']):
      companyInfo['confirmedLocations'] = company['confirmedLocations'][0]
      company_df = pd.json_normalize(companyInfo)
    else:
      company_df = pd.DataFrame(companyInfo, index=[i])
      company_df[['confirmedLocations.country',
        'confirmedLocations.geographicArea', 'confirmedLocations.city',
        'confirmedLocations.postalCode', 'confirmedLocations.description',
        'confirmedLocations.streetAddressOptOut',
        'confirmedLocations.headquarter', 'confirmedLocations.line2',
        'confirmedLocations.line1']] = np.nan
  else:
    company_df = pd.DataFrame()
    company_df[['name', 'entityUrn', 'confirmedLocations.country',
        'confirmedLocations.geographicArea', 'confirmedLocations.city',
        'confirmedLocations.postalCode', 'confirmedLocations.description',
        'confirmedLocations.streetAddressOptOut',
        'confirmedLocations.headquarter', 'confirmedLocations.line2',
        'confirmedLocations.line1']] = np.nan
  return(company_df)

### Frst Time to request data

In [ ]:
state_names = ["California", "Pennsylvania","Florida"]
               #"Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "Colorado", "Connecticut", "District of Columbia", "Delaware", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon",  "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
position_list = ["data scientist", "data analyst", "data engineer","gis analyst"]

In [ ]:
jobPost = pd.DataFrame()
for position in position_list:
  for state in state_names:
    df = get_jobPost_by_SatePosition(position, state)
    df['state'] = state
    df['position'] = position
    jobPost = pd.concat([jobPost,df])
    print(state)
  print("--------------Finished {position}--------------")

California
Pennsylvania
Florida
--------------Finished {}-------------- data scientist
California
Pennsylvania
Florida
--------------Finished {}-------------- data analyst
California
Pennsylvania
Florida
--------------Finished {}-------------- data engineer
California
Pennsylvania
Florida
--------------Finished {}-------------- gis analyst


Request and add company name.

In [ ]:
df = jobPost.copy()

In [ ]:
com_df = pd.DataFrame()
for i in range(len(df)):
  result = get_company_info(df.iloc[i])
  com_df = pd.concat([com_df, result])

df = pd.merge(df, com_df, how='left', right_on='entityUrn', left_on='companyDetails.company')

In [ ]:
df['recency'] = pd.to_datetime("today").date()
df.head()

,dashEntityUrn,formattedLocation,listedAt,title,$recipeTypes,$type,entityUrn_x,workRemoteAllowed,companyDetails.company,companyDetails.*companyResolutionResult,...,confirmedLocations.country,confirmedLocations.geographicArea,confirmedLocations.city,confirmedLocations.postalCode,confirmedLocations.description,confirmedLocations.streetAddressOptOut,confirmedLocations.headquarter,confirmedLocations.line2,confirmedLocations.line1,recency
0,urn:li:fsd_jobPosting:3010225380,United States,1.651310e+12,Software Developer,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,urn:li:fs_normalized_jobPosting:3010225380,True,urn:li:fs_normalized_company:13990,urn:li:fs_normalized_company:13990,...,US,Washington,Seattle,98101,Headquarters,False,1.0,Floor 31,1301 Second Avenue,2022-05-01
1,urn:li:fsd_jobPosting:3010225380,United States,1.651310e+12,Software Developer,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,urn:li:fs_normalized_jobPosting:3010225380,True,urn:li:fs_normalized_company:13990,urn:li:fs_normalized_company:13990,...,US,Washington,Seattle,98101,Headquarters,False,1.0,Floor 31,1301 Second Avenue,2022-05-01
2,urn:li:fsd_jobPosting:3010225380,United States,1.651310e+12,Software Developer,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,urn:li:fs_normalized_jobPosting:3010225380,True,urn:li:fs_normalized_company:13990,urn:li:fs_normalized_company:13990,...,US,Washington,Seattle,98101,Headquarters,False,1.0,Floor 31,1301 Second Avenue,2022-05-01
3,urn:li:fsd_jobPosting:3010225380,United States,1.651310e+12,Software Developer,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,urn:li:fs_normalized_jobPosting:3010225380,True,urn:li:fs_normalized_company:13990,urn:li:fs_normalized_company:13990,...,US,Washington,Seattle,98101,Headquarters,False,1.0,Floor 31,1301 Second Avenue,2022-05-01
4,urn:li:fsd_jobPosting:3010225380,United States,1.651310e+12,Software Developer,[com.linkedin.voyager.deco.jserp.WebSearchJobJ...,com.linkedin.voyager.jobs.JobPosting,urn:li:fs_normalized_jobPosting:3010225380,True,urn:li:fs_normalized_company:13990,urn:li:fs_normalized_company:13990,...,US,Washington,Seattle,98101,Headquarters,False,1.0,Floor 31,1301 Second Avenue,2022-05-01


### Second time to add more data

load clean data after first extraction.

In [144]:
df2 = pd.read_csv('https://raw.githubusercontent.com/keeea/Knowledge-Oriented-Service-Jobs-Dashboard/main/data/data.csv')

Extract map from company urn to name.

In [10]:
company_name = df2[['dashEntityUrn','name']].copy().drop_duplicates()

In [8]:
state_names2 = ["District of Columbia","Massachusetts", "North Carolina","New Jersey","Texas","Washington",]
               #"California", "Pennsylvania","Florida", "Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "Colorado", "Connecticut",  "Delaware", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana",  "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana",  "North Dakota", "Nebraska", "New Hampshire",  "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon",  "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee",  "Utah", "Virginia", "Virgin Islands", "Vermont",  "Wisconsin", "West Virginia", "Wyoming"]
position_list = ["data scientist", "data analyst", "data engineer","gis analyst"]

In [11]:
jobPost2 = pd.DataFrame()
for position in position_list:
  for state in state_names2:
    df = get_jobPost_by_SatePosition(position, state)
    df['state'] = state
    df['position'] = position
    jobPost2 = pd.concat([jobPost2,df])
    print(state)
  print("--------------Finished {position}--------------")

District of Columbia
Massachusetts
North Carolina
New Jersey
Texas
Washington
--------------Finished {position}--------------
District of Columbia
Massachusetts
North Carolina
New Jersey
Texas
Washington
--------------Finished {position}--------------
District of Columbia
Massachusetts
North Carolina
New Jersey
Texas
Washington
--------------Finished {position}--------------
District of Columbia
Massachusetts
North Carolina
New Jersey
Texas
Washington
--------------Finished {position}--------------


In [146]:
df3 = jobPost2.copy()

Obtain the unique urn list.

In [83]:
urn_list = df3[['companyDetails.company']].drop_duplicates()

In [106]:
com_df = pd.DataFrame()
for i in range(len(urn_list)):
  result = get_company_info(urn_list.iloc[i])
  com_df = pd.concat([com_df, result])

In [147]:
df3 = pd.merge(df3, com_df, how='left', right_on='entityUrn', left_on='companyDetails.company')

In [148]:
df3['recency'] = "5/13/22"
df2['recency'] = "5/1/22"

In [149]:
df3['entry'] = list(range(40000,40000+len(df3)))
df3 = df3[df2.columns]
final_df = pd.concat([df2,df3])

In [152]:
final_df.shape

(6463, 8)

In [155]:
final_df.dropna(subset=['name'], inplace=True)

Output to json format.

In [156]:
final_df.to_json(r'data', orient='records')

# U.S. BUREAU OF LABOR STATISTICS

In [ ]:
url1 = "https://raw.githubusercontent.com/keeea/Knowledge-Oriented-Service-Jobs-Dashboard/main/BLS_data/0_TCM.csv"
url2 = "https://raw.githubusercontent.com/keeea/Knowledge-Oriented-Service-Jobs-Dashboard/main/BLS_data/1_TFP.csv"
url3 = "https://raw.githubusercontent.com/keeea/Knowledge-Oriented-Service-Jobs-Dashboard/main/BLS_data/2_ELG.csv"

In [ ]:
df1 = pd.read_csv(url1).dropna(how='all')
df2 = pd.read_csv(url2).dropna(how='all')
df3 = pd.read_csv(url3).dropna(how='all')

In [ ]:
df1 = df1.rename(columns={"Unnamed: 0":"state"})
df2 = df2.rename(columns={"Unnamed: 0":"state"})
df3 = df3.rename(columns={"Unnamed: 0":"state"})
df3.head()

,state,Mar-21,Jan-22,Feb-22,Mar-22,Mar-21.1,Jan-22.1,Feb-22.1,Mar-22.1,Mar-21.2,Jan-22.2,Feb-22.2,Mar-22.2
0,Alabama,240.1,238.9,239.4,239.9,183.5,192.4,192.8,193.3,386,386.8,385.7,386.6
1,Alaska,50.8,50.5,50.2,50,30.5,32.7,33.2,33.2,77,77.4,77,77
2,Arizona,461.8,470.1,471.8,473,285.2,325,325.9,321.2,402.4,406.6,405.8,405.5
3,Arkansas,196.6,194.7,195.5,194.1,111.8,120.4,121,121.9,206.3,208.9,208.9,208.7
4,California,"2,776.10","2,864.00","2,882.00","2,891.00","1,447.20","1,816.90","1,842.70","1,857.50","2,423.80","2,503.60","2,505.40","2,505.60"


In [ ]:
from pandas._libs.tslibs import dtypes
df1.dtypes

state        object
Mar-21       object
Jan-22       object
Feb-22       object
Mar-22       object
Mar-21.1    float64
Jan-22.1    float64
Feb-22.1    float64
Mar-22.1    float64
Mar-21.2     object
Jan-22.2     object
Feb-22.2     object
Mar-22.2     object
dtype: object

In [ ]:
df3.dtypes

state       object
Mar-21      object
Jan-22      object
Feb-22      object
Mar-22      object
Mar-21.1    object
Jan-22.1    object
Feb-22.1    object
Mar-22.1    object
Mar-21.2    object
Jan-22.2    object
Feb-22.2    object
Mar-22.2    object
dtype: object

In [ ]:
for col in ['Feb-22','Mar-22','Feb-22.1','Mar-22.1','Feb-22.2','Mar-22.2']:
  df3[col] = df3[col].str.replace(',', '').astype(float)

for col in ['Feb-22','Mar-22','Feb-22.2','Mar-22.2']:
  df1[col] = df1[col].str.replace(',', '').astype(float)

In [ ]:
df1['Total'] = (df1['Mar-22'] - df1['Feb-22']) / df1['Feb-22'] *100
df1['Construction'] = (df1['Mar-22.1'] - df1['Feb-22.1']) / df1['Feb-22.1'] *100
df1['Manufacturing'] = (df1['Mar-22.2'] - df1['Feb-22.2']) / df1['Feb-22.2'] *100
df1.head()

,state,Mar-21,Jan-22,Feb-22,Mar-22,Mar-21.1,Jan-22.1,Feb-22.1,Mar-22.1,Mar-21.2,Jan-22.2,Feb-22.2,Mar-22.2,Total,Construction,Manufacturing
0,Alabama,"2,030.50","2,055.80",2064.1,2066.5,93.5,98.7,98.0,98.9,263.2,266,267.9,266.8,0.116273,0.918367,-0.410601
1,Alaska,310.7,316.9,315.8,315.7,15.8,16.3,16.5,16.7,12,12.9,11.3,11.2,-0.031666,1.212121,-0.884956
2,Arizona,"2,920.70","3,022.30",3033.2,3029.5,177.7,181.0,181.6,182.6,180.3,184.7,186.1,186.8,-0.121983,0.550661,0.376142
3,Arkansas,"1,273.20","1,309.00",1308.8,1308.5,55.1,55.6,55.2,55.5,157.1,161,161.2,161.7,-0.022922,0.543478,0.310174
4,California,"16,354.50","17,200.80",17336.2,17396.4,885.3,885.5,907.0,915.9,"1,268.50","1,288.60",1294.4,1297.3,0.347250,0.981257,0.224042


In [ ]:
df2['Trade_transportation_utilities'] = (df2['Mar-22'] - df2['Feb-22']) / df2['Feb-22'] *100
df2['FinancialActivities'] = (df2['Mar-22.1'] - df2['Feb-22.1']) / df2['Feb-22.1'] *100
df2['ProfessionalAndBusinessServices'] = (df2['Mar-22.2'] - df2['Feb-22.2']) / df2['Feb-22.2'] *100
df2.head()

,state,Mar-21,Jan-22,Feb-22,Mar-22,Mar-21.1,Jan-22.1,Feb-22.1,Mar-22.1,Mar-21.2,Jan-22.2,Feb-22.2,Mar-22.2,Trade_transportation_utilities,FinancialActivities,ProfessionalAndBusinessServices
0,Alabama,2030.5,2055.8,2064.1,2066.5,93.5,98.7,98.0,98.9,263.2,266.0,267.9,266.8,0.116273,0.918367,-0.410601
1,Alaska,310.7,316.9,315.8,315.7,15.8,16.3,16.5,16.7,12.0,12.9,11.3,11.2,-0.031666,1.212121,-0.884956
2,Arizona,2920.7,3022.3,3033.2,3029.5,177.7,181.0,181.6,182.6,180.3,184.7,186.1,186.8,-0.121983,0.550661,0.376142
3,Arkansas,1273.2,1309.0,1308.8,1308.5,55.1,55.6,55.2,55.5,157.1,161.0,161.2,161.7,-0.022922,0.543478,0.310174
4,California,16354.5,17200.8,17336.2,17396.4,885.3,885.5,907.0,915.9,1268.5,1288.6,1294.4,1297.3,0.347250,0.981257,0.224042


In [ ]:
df3['EducationAndHealthServices'] = (df3['Mar-22'] - df3['Feb-22']) / df3['Feb-22'] *100
df3['LeisureAndHospitality'] = (df3['Mar-22.1'] - df3['Feb-22.1']) / df3['Feb-22.1'] *100
df3['Government'] = (df3['Mar-22.2'] - df3['Feb-22.2']) / df3['Feb-22.2'] *100
df3.head()

,state,Mar-21,Jan-22,Feb-22,Mar-22,Mar-21.1,Jan-22.1,Feb-22.1,Mar-22.1,Mar-21.2,Jan-22.2,Feb-22.2,Mar-22.2,EducationAndHealthServices,LeisureAndHospitality,Government
0,Alabama,240.1,238.9,239.4,239.9,183.5,192.4,192.8,193.3,386,386.8,385.7,386.6,0.208855,0.259336,0.233342
1,Alaska,50.8,50.5,50.2,50.0,30.5,32.7,33.2,33.2,77,77.4,77.0,77.0,-0.398406,0.000000,0.000000
2,Arizona,461.8,470.1,471.8,473.0,285.2,325,325.9,321.2,402.4,406.6,405.8,405.5,0.254345,-1.442160,-0.073928
3,Arkansas,196.6,194.7,195.5,194.1,111.8,120.4,121.0,121.9,206.3,208.9,208.9,208.7,-0.716113,0.743802,-0.095740
4,California,"2,776.10","2,864.00",2882.0,2891.0,"1,447.20","1,816.90",1842.7,1857.5,"2,423.80","2,503.60",2505.4,2505.6,0.312283,0.803169,0.007983


In [ ]:
df1_cl = df1.iloc[:,[0,13,14,15]]
df2_cl = df2.iloc[:,[0,13,14,15]]
df3_cl = df3.iloc[:,[0,13,14,15]]

In [ ]:
df_final = pd.merge(df1_cl, df2_cl, on='state')
df_final = pd.merge(df_final, df3_cl, on='state')
df_final.head()

,state,Total,Construction,Manufacturing,Trade_transportation_utilities,FinancialActivities,ProfessionalAndBusinessServices,EducationAndHealthServices,LeisureAndHospitality,Government
0,Alabama,0.116273,0.918367,-0.410601,0.116273,0.918367,-0.410601,0.208855,0.259336,0.233342
1,Alaska,-0.031666,1.212121,-0.884956,-0.031666,1.212121,-0.884956,-0.398406,0.000000,0.000000
2,Arizona,-0.121983,0.550661,0.376142,-0.121983,0.550661,0.376142,0.254345,-1.442160,-0.073928
3,Arkansas,-0.022922,0.543478,0.310174,-0.022922,0.543478,0.310174,-0.716113,0.743802,-0.095740
4,California,0.347250,0.981257,0.224042,0.347250,0.981257,0.224042,0.312283,0.803169,0.007983


In [ ]:
state = gpd.read_file('https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json')

In [ ]:
state = state.drop(columns='id')
df_final = pd.merge(df_final, state, how='left', left_on='state', right_on='name')
df_final.head()

,state,Total,Construction,Manufacturing,Trade_transportation_utilities,FinancialActivities,ProfessionalAndBusinessServices,EducationAndHealthServices,LeisureAndHospitality,Government,name,density,geometry
0,Alabama,0.116273,0.918367,-0.410601,0.116273,0.918367,-0.410601,0.208855,0.259336,0.233342,Alabama,94.650,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."
1,Alaska,-0.031666,1.212121,-0.884956,-0.031666,1.212121,-0.884956,-0.398406,0.000000,0.000000,Alaska,1.264,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691..."
2,Arizona,-0.121983,0.550661,0.376142,-0.121983,0.550661,0.376142,0.254345,-1.442160,-0.073928,Arizona,57.050,"POLYGON ((-109.04250 37.00026, -109.04798 31.3..."
3,Arkansas,-0.022922,0.543478,0.310174,-0.022922,0.543478,0.310174,-0.716113,0.743802,-0.095740,Arkansas,56.430,"POLYGON ((-94.47384 36.50186, -90.15254 36.496..."
4,California,0.347250,0.981257,0.224042,0.347250,0.981257,0.224042,0.312283,0.803169,0.007983,California,241.700,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."


In [ ]:
df_final = df_final.drop(columns = 'name')

In [ ]:
df_final = gpd.GeoDataFrame(df_final, geometry='geometry')
df_final.to_file("employment_change.GeoJSON")

In [ ]:
df_final.describe()

,Total,Construction,Manufacturing,Trade_transportation_utilities,FinancialActivities,ProfessionalAndBusinessServices,EducationAndHealthServices,LeisureAndHospitality,Government,density
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,0.155709,0.434352,0.154809,0.155709,0.434352,0.154809,0.108786,0.319787,0.010488,207.894469
std,0.226301,1.533129,0.561350,0.226301,1.533129,0.561350,0.474643,0.869630,0.250364,288.772752
min,-0.350140,-3.258656,-1.617586,-0.350140,-3.258656,-1.617586,-1.379310,-2.004295,-0.748299,1.264000
25%,-0.022922,-0.048709,-0.071608,-0.022922,-0.048709,-0.071608,-0.065703,-0.152439,-0.109469,49.330000
50%,0.161512,0.445186,0.251256,0.161512,0.445186,0.251256,0.142857,0.228751,0.014795,98.070000
75%,0.306220,1.046798,0.458015,0.306220,1.046798,0.458015,0.342466,0.803169,0.181283,204.500000
max,0.586775,7.171315,1.360430,0.586775,7.171315,1.360430,1.168142,3.348554,0.592417,1189.000000
